In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image size and path to the dataset
img_size = (512, 512)
path = "D:\\Casper\\OTHER\\Data\\dpaml_hw3\\face_data"

# Define different data augmentations
datagen = ImageDataGenerator(rescale=1./255)
datagen_flip_v = ImageDataGenerator(
    rescale=1./255,
    vertical_flip=True
)
datagen_flip_h = ImageDataGenerator(
    rescale=1./255,
    horizontal_flip=True
)
datagen_flip_both = ImageDataGenerator(
    rescale=1./255,
    vertical_flip=True,
    horizontal_flip=True
)

# Create data generators
data_flow = datagen.flow_from_directory(path, target_size=img_size, class_mode='categorical', batch_size=32, shuffle=True)
data_flow_v = datagen_flip_v.flow_from_directory(path, target_size=img_size, class_mode='categorical', batch_size=32, shuffle=True)
data_flow_h = datagen_flip_h.flow_from_directory(path, target_size=img_size, class_mode='categorical', batch_size=32, shuffle=True)
data_flow_both = datagen_flip_both.flow_from_directory(path, target_size=img_size, class_mode='categorical', batch_size=32, shuffle=True)


Found 310 images belonging to 4 classes.
Found 310 images belonging to 4 classes.
Found 310 images belonging to 4 classes.
Found 310 images belonging to 4 classes.


In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models, Model

class SimpleCNN_a(tf.keras.Model):
    def __init__(self):
        super(SimpleCNN_a, self).__init__()
        self.conv1 = layers.Conv2D(20, (3, 3), strides=2)
        # Pooling is commented out in the original PyTorch code, we'll skip it here too
        self.conv2 = layers.Conv2D(40, (3, 3), strides=2)
        self.conv3 = layers.Conv2D(20, (3, 3), strides=2)
        self.conv4 = layers.Conv2D(1, (3, 3), strides=2)
        self.flatten = layers.Flatten()
        # Inferred from PyTorch code - assuming the feature size here, adjust as needed
        self.fc = layers.Dense(4)

    def call(self, x):
        x = self.conv1(x)
        x = tf.nn.relu(x)
        x = self.conv2(x)
        x = tf.nn.relu(x)
        x = self.conv3(x)
        x = tf.nn.relu(x)
        x = self.conv4(x)
        x = tf.nn.relu(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

class SimpleCNN_b(tf.keras.Model):
    def __init__(self):
        super(SimpleCNN_b, self).__init__()
        self.conv1 = layers.Conv2D(50, (3, 3), strides=2)
        self.pool = layers.MaxPooling2D(2)
        self.conv2 = layers.Conv2D(1, (3, 3), strides=2)
        self.flatten = layers.Flatten()
        self.fc = layers.Dense(4)

    def call(self, x):
        x = self.conv1(x)
        x = tf.nn.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = tf.nn.relu(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.fc(x)
        return x

# Creating an instance of the second model and testing it
model = SimpleCNN_b()

# Creating a random input tensor
img = tf.random.normal([2, 512, 512, 3])  # Note TensorFlow uses NHWC format by default
output = model(img)
print(output.shape)  # Should print (2, 4)


(2, 4)


In [2]:
import tensorflow as tf
from datetime import datetime
import os

def pprint(output='\n', show_time=False):
    filename = "hw3-APR18.txt"
    print(output)
    with open(filename, 'a') as f:
        if show_time:
            f.write(datetime.now().strftime("[%Y-%m-%d %H:%M:%S] "))
        f.write(str(output))
        f.write('\n')


In [4]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy, TopKCategoricalAccuracy
import time
import numpy as np

def train(model_lists, model_name, loaders, num_class, phases=['train'], epochs=20, save_weight=False):
    model = model_lists[model_name]()
    
    # Adjust the model based on its name
    if model_name in ["resnet18", "mobileNet_v2"]:
        num_ftrs = model.output_shape[-1]  # Assuming model has been built and output shape is available
        model.layers[-1] = tf.keras.layers.Dense(num_class, activation='softmax')

    pprint(f"Training {model_name}", True)
    
    # Count parameters (sum of all trainable and non-trainable variables)
    model_parameters_amount = np.sum([np.prod(v.shape) for v in model.trainable_variables])
    pprint(f"model total parameters: {model_parameters_amount:,}")

    # Define the optimizer, loss, and metrics
    optimizer = Adam(learning_rate=0.005)
    loss_fn = SparseCategoricalCrossentropy()
    train_acc_metric = SparseCategoricalAccuracy()
    top3_acc_metric = TopKCategoricalAccuracy(k=3)

    pprint(f"learning rate={0.005}")
    start = time.time()

    # Train and validation phases
    for epoch in range(epochs):
        for phase in phases:
            if phase == 'train':
                model.trainable = True
            else:
                model.trainable = False

            running_loss = 0.0
            correct_predictions = 0
            total_samples = 0

            for images, labels in loaders[phase]:
                with tf.GradientTape() as tape:
                    logits = model(images, training=(phase == 'train'))
                    loss = loss_fn(labels, logits)

                if phase == 'train':
                    grads = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(grads, model.trainable_variables))
                
                # Update metrics
                train_acc_metric.update_state(labels, logits)
                top3_acc_metric.update_state(labels, logits)
                
                running_loss += loss.numpy() * images.shape[0]
                total_samples += images.shape[0]

            avg_loss = running_loss / total_samples
            top1_accuracy = train_acc_metric.result().numpy() * 100
            top3_accuracy = top3_acc_metric.result().numpy() * 100

            train_acc_metric.reset_states()
            top3_acc_metric.reset_states()
            
            pprint(f"Epoch [{epoch+1}/{epochs}], phase: {phase}, samples: {total_samples}, Loss: {avg_loss:.4f}, Top-1 Accuracy: {top1_accuracy:.2f}%, Top-3 Accuracy: {top3_accuracy:.2f}%")
    
    duration = time.time() - start
    pprint(f"Elapsed time: {duration} seconds")
    
    if save_weight:
        model.save(f'{model_name}.h5')
        pprint(f"weight saved as: {model_name}.h5")

model_list ={
    "resnet18": lambda: models.resnet18(),
    "mobileNet_v2": lambda: models.mobilenet_v2(),
    "mod_face_a": lambda: mod_resnet(BasicBlock, [2, 2, 0, 0], channel_num_list=[4, 16, 16, ], num_classes=4, input_channel=3),
    "mod_face_b": lambda: mod_resnet(BasicBlock, [2, 2, 0, 0], channel_num_list=[4, 8, 8, ], num_classes=4, input_channel=3),
    "SimpleCNN_a": lambda: SimpleCNN_a(),
    "SimpleCNN_b": lambda: SimpleCNN_b(),
}
model_names = [

    # "resnet18", # 11,178,051
    # "mobileNet_v2", # 2,227,715
    # "mod_face_a", # 17,991
    # "mod_face_b", # 5,199
    # "SimpleCNN_a", # 19,049
    "SimpleCNN_b", # 4,737
]

for ii in range(len(model_names)):
    model_name = model_names[ii]
    phases = ['train', 'val']
    train(model_list, model_name, data_loaders, 4, phases, epochs=1, save_weight=True)

NameError: name 'data_loaders' is not defined

In [7]:
class BasicBlock(Model):
    expansion = 1

    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = layers.Conv2D(out_channels, kernel_size=3, strides=stride, padding='same', use_bias=False)
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(out_channels, kernel_size=3, padding='same', use_bias=False)
        self.bn2 = layers.BatchNormalization()
        self.downsample = downsample
        self.relu = layers.ReLU()

    def call(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

class ResNet(Model):
    def __init__(self, block, layer_list, channel_list, num_classes=1000, input_shape=(224, 224, 3)):
        super(ResNet, self).__init__()
        self.conv1 = layers.Conv2D(channel_list[0], kernel_size=7, strides=2, padding='same', input_shape=input_shape, use_bias=False)
        self.bn1 = layers.BatchNormalization()
        self.relu = layers.ReLU()
        self.maxpool = layers.MaxPooling2D(pool_size=3, strides=2, padding='same')
        self.layer1 = self._make_layer(block, channel_list[1], layer_list[0])
        self.layer2 = self._make_layer(block, channel_list[2], layer_list[1], stride=2)
        self.layer3 = self._make_layer(block, channel_list[3], layer_list[2], stride=2)
        self.layer4 = self._make_layer(block, channel_list[4], layer_list[3], stride=2)
        self.avgpool = layers.GlobalAveragePooling2D()
        self.fc = layers.Dense(num_classes)

    def call(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = self.fc(x)
        return x

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1:
            downsample = tf.keras.Sequential([
                layers.Conv2D(planes * block.expansion, kernel_size=1, strides=stride, use_bias=False),
                layers.BatchNormalization()
            ])

        layers_ = []
        layers_.append(block(self.in_channels, planes, stride, downsample))
        self.in_channels = planes * block.expansion
        for _ in range(1, blocks):
            layers_.append(block(self.in_channels, planes))

        return tf.keras.Sequential(layers_)
# Create a ResNet model instance
model = ResNet(BasicBlock, [2, 2, 2, 2], num_classes=1000)
# Dummy input tensor
input_tensor = tf.random.normal([1, 224, 224, 3])
# Forward pass
output = model(input_tensor)
print(output.shape)  # Output shape should be (1, 1000) for 1000 classes


AttributeError: 'ResNet' object has no attribute 'in_channels'